# Complete Solution

This notebook that analysis the Crown.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import os
import glob
import pathlib
import numpy as np
import pandas as pd
import open3d as o3d
from tqdm import tqdm

import tree as tree_utils
import utils.o3d_utils as o3d_utils
import utils.ahn_utils as ahn_utils
import utils.las_utils as las_utils

#### Process File
----

In [ ]:
adTree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'

# Load point cloud data. source can be: ahn / cyclo / sonarski
las_file = '../../../datasets/Waternet/clipped/cyclo_119287_484147_Ulmus_4.las'
tree_cloud = o3d_utils.read_las(las_file)

# surface reconstruction
ahn_data_folder = '../../../datasets/Waternet/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)
treecode = las_utils.get_treecode_from_filename(las_file)
ground_points = npz_reader.get_tree_surface(treecode)
ground_cloud = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(ground_points))

In [ ]:
o3d.visualization.draw_geometries([ground_cloud,tree_cloud])

In [ ]:
# Analyse tree
result = tree_utils.process_tree(tree_cloud, ground_cloud, adTree_exe, crown_model_method='Convex_Hull')

In [ ]:
o3d.visualization.draw_geometries([result['LoD']])

#### Process Folder
----

In [ ]:
def process_file(file, npz_reader, adTree_exe):

    tree_cloud = o3d_utils.read_las(file)
    treecode = las_utils.get_treecode_from_filename(file.name)
    ground_points = npz_reader.get_tree_surface(treecode)
    ground_cloud = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(ground_points))

    processed_tree = tree_utils.process_tree(tree_cloud, ground_cloud, adTree_exe, crown_model_method='Convex_Hull')

    return processed_tree


def process_folder(in_folder, npz_reader, adTree_exe):

    if not os.path.isdir(in_folder):
        print('The input path specified does not exist')

    if type(in_folder) == str:
        in_folder = pathlib.Path(in_folder)

    file_types = ('.LAS', '.las', '.LAZ', '.laz')
    files = [f for f in in_folder.glob('*')
                if f.name.endswith(file_types)]

    files_tqdm = tqdm(files, unit="file",
                    disable=False, smoothing=0)

    results = []
    for file in files:
        print(file.name)
        result = process_file(file, npz_reader=npz_reader, adTree_exe=adTree_exe)
        result = {key:result[key] for key in ['stem_startpoint', 'stem_height', 'stem_angle', 'DBH', 'crown_baseheight', 'crown_height', 'crown_diameter', 'crown_volume', 'crown_volume']}
        result['Boom'] = file.name
        results.append(result)

    df = pd.DataFrame(results)
    return df

In [ ]:
in_folder = '../../../datasets/Waternet/clipped/'
ahn_data_folder = '../../../datasets/Waternet/ahn_surf/'
# out_folder = '../run1/'

adTree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'

npz_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
df = process_folder(in_folder, npz_reader, adTree_exe)